In [1]:
'''
## 共享GPU
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.25)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
'''

'\n## 共享GPU\nimport tensorflow as tf\ngpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.25)\nsess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))\n'

In [2]:
#数据读入并展示
import string
import numpy as np

f = open('/mnt/data/poems_clean.txt',"r",encoding='utf-8')
poems = []
for line in f.readlines():  
     title, poem = line.split(':')     #去标题
     poem = poem.replace(' ', '')      #去空格
     poem = poem.replace('\n', '')
     poems.append(list(poem))
print(poems[0][:])

['寒', '随', '穷', '律', '变', '春', '逐', '鸟', '声', '开', '初', '风', '飘', '带', '柳', '晚', '雪', '间', '花', '梅', '碧', '林', '青', '旧', '竹', '绿', '沼', '翠', '新', '苔', '芝', '田', '初', '雁', '去', '绮', '树', '巧', '莺', '来']


In [3]:
#Tokenizer编码
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems)
poems_digit = tokenizer.texts_to_sequences(poems)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

Using TensorFlow backend.


5546

In [4]:
#处理“长短不一”
poems_digit = pad_sequences(poems_digit, maxlen=50, padding='post')
print("原始诗歌")
print(poems[3864])
print("\n")
print("编码+补全后的结果")
print(poems_digit[3864])

原始诗歌
['床', '前', '明', '月', '光', '疑', '是', '地', '上', '霜', '举', '头', '望', '明', '月', '低', '头', '思', '故', '乡']


编码+补全后的结果
[532  72  53  13 140 429  44 113  15 202 688 128 106  53  13 502 128  75
 134 169   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [5]:
#矩阵拆分
X = poems_digit[:, :-1]
Y = poems_digit[:, 1:]

print(poems_digit.shape)
print(X.shape)
print(Y.shape)

print("X示例", "\t", "Y示例")

for i in range(10):
     print(X[0][i], "\t", Y[0][i])
    
print("...", "\t", "...")

(24117, 50)
(24117, 49)
(24117, 49)
X示例 	 Y示例
42 	 180
180 	 401
401 	 1143
1143 	 671
671 	 9
9 	 331
331 	 130
130 	 58
58 	 84
84 	 177
... 	 ...


In [6]:
#one-hot编码
print(vocab_size)
from keras.utils import to_categorical
Y = to_categorical(Y, num_classes=vocab_size)
print(Y.shape)

5546
(24117, 49, 5546)


In [7]:
#定义虚拟空间维度
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, Embedding, Activation, BatchNormalization
embedding_size = 64
hidden_size = 128

In [8]:
#RNN模型代码
inp = Input(shape=(49,))
x = Embedding(vocab_size, embedding_size, mask_zero=True)(inp)
x = SimpleRNN(hidden_size,return_sequences=True)(x)
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 49)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 49, 64)            354944    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 49, 128)           24704     
_________________________________________________________________
dense_1 (Dense)              (None, 49, 5546)          715434    
_________________________________________________________________
activation_1 (Activation)    (None, 49, 5546)          0         
Total params: 1,095,082
Trainable params: 1,095,082
Non-trainable params: 0
_________________________________________________________________


In [9]:
#模型编译与拟合
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X, Y, epochs=10, batch_size=128, validation_split=0.2)

/root/miniconda3/envs/myconda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 19293 samples, validate on 4824 samples
Epoch 1/10
19293/19293 [==============================] - 45s 2ms/step - loss: 4.9130 - accuracy: 0.0279 - val_loss: 4.4623 - val_accuracy: 0.0321
Epoch 2/10
19293/19293 [==============================] - 39s 2ms/step - loss: 4.7793 - accuracy: 0.0296 - val_loss: 4.4572 - val_accuracy: 0.0321
Epoch 3/10
19293/19293 [==============================] - 38s 2ms/step - loss: 4.7584 - accuracy: 0.0297 - val_loss: 4.4374 - val_accuracy: 0.0323
Epoch 4/10
19293/19293 [==============================] - 38s 2ms/step - loss: 4.7252 - accuracy: 0.0302 - val_loss: 4.4017 - val_accuracy: 0.0323
Epoch 5/10
19293/19293 [==============================] - 37s 2ms/step - loss: 4.6898 - accuracy: 0.0317 - val_loss: 4.3747 - val_accuracy: 0.0346
Epoch 6/10
19293/19293 [==============================] - 38s 2ms/step - loss: 4.6330 - accuracy: 0.0354 - val_loss: 4.3167 - val_accuracy: 0.0379
Epoch 7/10
19293/19293 [==============================] - 38s 2ms/ste

In [10]:
#RNN做藏头诗
poem_incomplete = '佳******期******如******梦******'
poem_index=[]
poem_text=''
for i in range(len(poem_incomplete)):
    current_word=poem_incomplete[i]
    
    if current_word!='*':
        index=tokenizer.word_index[current_word]
        
    else:
        x=np.expand_dims(poem_index,axis=0)
        x=pad_sequences(x,maxlen=49,padding='post')
        y=model.predict(x)[0,i]
        
        y[0]=0
        index=y.argmax()
        current_word=tokenizer.index_word[index]
        
    poem_index.append(index)
    poem_text=poem_text+current_word
    
poem_text=poem_text[0:]
print(poem_text[0:7])
print(poem_text[7:14])
print(poem_text[14:21])
print(poem_text[21:28])

佳人不可见何处
期何处不见人何
如不得不知人不
梦无人去何人不


In [15]:
#RNN做藏头诗
poem_incomplete = '佳****期****如****梦****'
poem_index=[]
poem_text=''
for i in range(len(poem_incomplete)):
    current_word=poem_incomplete[i]
    
    if current_word!='*':
        index=tokenizer.word_index[current_word]
        
    else:
        x=np.expand_dims(poem_index,axis=0)
        x=pad_sequences(x,maxlen=49,padding='post')
        y=model.predict(x)[0,i]
        
        y[0]=0
        index=y.argmax()
        current_word=tokenizer.index_word[index]
        
    poem_index.append(index)
    poem_text=poem_text+current_word
    
poem_text=poem_text[0:]
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

佳人不可见
期子不相逢
如此人何处
梦来不可知
